## Source: Wikipedia
## Retriever: Google Search
## Model: Roberta Large BoolQ

In [2]:
import torch
from sklearn.metrics import classification_report, roc_auc_score

import sys
sys.path.append("../../") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc
import pandas as pd
from progressbar import progressbar as pb

In [3]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

model_name = "apugachev/roberta-large-boolq-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
print("OK")

OK


In [4]:
def predict(question, passage, device=device):
    sequence = tokenizer.encode_plus(
        question, 
        passage, 
        return_tensors="pt",
        max_length=tokenizer.model_max_length,
        truncation=True
    )['input_ids']

    logits = model(sequence.to(device))[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = probabilities[1]
    
    return proba_yes

In [17]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Query

In [18]:
raw_df = pd.read_csv("../../../data/google_search_wiki/wikipedia_articles_keywords.csv")
df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
    .dropna()
)
print(len(df))
df.head(1)

519


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
0,cranberries urinary tract infections,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['urinary', 'infection', 'tract']","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections


In [19]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (519 of 519) |######################| Elapsed Time: 0:14:29 Time:  0:14:29


In [20]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,cranberries urinary tract infections,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['urinary', 'infection', 'tract']","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000308


In [21]:
QUERY_SAVE_PRED_FILE = "predictions/roberta_large_boolq_google_search_wiki_preds_keywords.csv"

df.to_csv(QUERY_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [23]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [24]:
df = pd.read_csv(QUERY_SAVE_PRED_FILE)
print(len(df))
df.head(2)

519


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,cranberries urinary tract infections,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['urinary', 'infection', 'tract']","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000308
1,cranberries urinary tract infections,Cranberry juice,Cranberry juice is the liquid juice of the cra...,2.0,['cranberry'],1.0,"['urinary', 'infection', 'tract']","['urinary', 'infection', 'tract']",3.0,0.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000280


In [25]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [26]:
df_filled = (
    df
    .merge(init_data, how='outer', on=['query_id', 'data_source', 'label'])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

538


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206,misbelief,0.0,NaN,NaN,0.5,Does aloe help for a runny nose?,aloe help runny nose
534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212,misbelief,1.0,NaN,NaN,0.5,Does echinacea boost immunity?,echinacea boost immunity
535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213,misbelief,1.0,NaN,NaN,0.5,Does honey boost immunity?,honey boost immunity
536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214,misbelief,1.0,NaN,NaN,0.5,Does garlic kill viruses?,garlic kill viruses
537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_h,health_belief,1.0,NaN,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate


In [27]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [28]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.8443,0.8272,0.7500,0.6429,0.7869
top1,0.8097,0.8448,0.7500,0.7500,0.7981
norm_linear,0.8512,0.8512,0.9167,0.6250,0.8124
norm_log,0.8478,0.8656,0.9167,0.6250,0.8162


## Question

In [29]:
raw_df = pd.concat((
    pd.read_csv("../../../data/google_search_wiki/wikipedia_articles_question.csv"),
))
df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
    .dropna()
)
print(len(df))
df.head(1)

627


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
0,Can cranberries prevent urinary tract infections?,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['prevent', 'tract', 'urinary', 'can', 'infect...","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections


In [31]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (627 of 627) |######################| Elapsed Time: 0:22:03 Time:  0:22:03


In [32]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,Can cranberries prevent urinary tract infections?,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['prevent', 'tract', 'urinary', 'can', 'infect...","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000308


In [33]:
DESCRIPTION_SAVE_PRED_FILE = "predictions/roberta_large_boolq_google_search_wiki_preds_question.csv"

df.to_csv(DESCRIPTION_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [34]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [35]:
df = pd.read_csv(DESCRIPTION_SAVE_PRED_FILE)
print(len(df))
df.head(2)

627


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,Can cranberries prevent urinary tract infections?,Cranberry,A comprehensive review in 2012 of available re...,1.0,['cranberry'],1.0,"['prevent', 'tract', 'urinary', 'can', 'infect...","['urinary', 'infection', 'tract']",3.0,82.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000308
1,Can cranberries prevent urinary tract infections?,Cranberry,Cranberries are a group of evergreen dwarf shr...,1.0,['cranberry'],1.0,"['prevent', 'tract', 'urinary', 'can', 'infect...",['can'],1.0,0.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.000884


In [36]:
df_filled = (
    df
    .merge(init_data, how='outer', on=['query_id', 'data_source', 'label'])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(5)

643


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205,misbelief,0.0,NaN,NaN,0.5,Does garlic help with thrush?,garlic help thrush
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206,misbelief,0.0,NaN,NaN,0.5,Does aloe help for a runny nose?,aloe help runny nose
640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212,misbelief,1.0,NaN,NaN,0.5,Does echinacea boost immunity?,echinacea boost immunity
641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213,misbelief,1.0,NaN,NaN,0.5,Does honey boost immunity?,honey boost immunity
642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_h,health_belief,1.0,NaN,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate


In [37]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.7093,0.8672,0.9167,0.5893,0.7522
top1,0.7059,0.8160,0.8333,0.8214,0.7544
norm_linear,0.6955,0.8768,0.9167,0.6250,0.7497
norm_log,0.6817,0.8704,0.9167,0.6429,0.7468
